In [ ]:
import requests
import pandas as pd
import time
!pip install tqdm

**A. Indie Games  Extraction from SteapSpy**

In [ ]:
import requests
import time
import json
from tqdm import tqdm
import pandas as pd

STEAMSPY_GENRE_URL = "https://steamspy.com/api.php?request=genre&genre=Indie"
STEAMSPY_APPDETAILS_URL = "https://steamspy.com/api.php?request=appdetails&appid={appid}"

# Step 1: Fetch all Indie games from SteamSpy API
def get_indie_games():
    response = requests.get(STEAMSPY_GENRE_URL)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch Indie games from SteamSpy.")
        return None

# Step 2: Get details for a specific game using SteamSpy
def get_steamspy_game_details(appid):
    url = STEAMSPY_APPDETAILS_URL.format(appid=appid)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch details for appid: {appid}")
        return None

# Step 3: Filter games by checking if 'Indie' is in their genre list and only include selected fields
def get_detailed_indie_games(indie_games):
    detailed_games = []
    for appid, game_data in tqdm(indie_games.items(), desc="Fetching game details"):
        time.sleep(1)  # Time sleep to respect the author's indications
        details = get_steamspy_game_details(appid)

        if details and details['appid'] != 999999 and 'Indie' in details.get('genre', []):
            game_info = {
                "appid": details.get("appid"),
                "name": details.get("name"),
                "developer": details.get("developer"),
                "publisher": details.get("publisher"),
                "initialprice": details.get("initialprice"),
                "average_forever": details.get("average_forever"),
                "owners": details.get("owners"),
                "languages": details.get("languages")
            }
            detailed_games.append(game_info)
    return detailed_games

def main():
    print("Fetching Indie games...")
    indie_games = get_indie_games()

    if indie_games:
        print(f"Total Indie games found: {len(indie_games)}")

        print("Fetching detailed information for each game...")
        detailed_games = get_detailed_indie_games(indie_games)

        print(f"Total games with detailed info retrieved and filtered by Indie genre: {len(detailed_games)}")

        df = pd.DataFrame(detailed_games)

        file_path = "/content/drive/My Drive/df_spy.xlsx"
        df.to_excel(file_path, index=False)
        print(f"Data saved to {file_path}")

if __name__ == "__main__":
    main()


Fetching Indie games...
Total Indie games found: 54750
Fetching detailed information for each game...


Fetching game details:   0%|          | 1/54750 [00:01<25:28:49,  1.68s/it]


KeyboardInterrupt: 

**B.  Getting tags and genre**

In [ ]:
import requests
import pandas as pd
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive')

# My personal Steam Key
STEAM_API_KEY = '1C1A5BFA2C3DBA9CEBBDF35081114F48'

# Step 2:Loading our file
file_path = '/content/drive/My Drive/df_spy.xlsx'
indie_games_df = pd.read_excel(file_path)

# Define Steam API URL for AppDetails
STEAM_APPDETAILS_URL = "https://store.steampowered.com/api/appdetails?appids={appid}&key={API_KEY}"

### Step 1: Fetch Tags and Genre from Steam API (appdetails endpoint)
def fetch_tags_and_genre(appid):
    url = STEAM_APPDETAILS_URL.format(appid=appid, API_KEY=STEAM_API_KEY)
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if str(appid) in data and data[str(appid)].get('success', False):
            app_data = data[str(appid)]['data']
            # Extract tags and genres
            tags = ', '.join(app_data.get('tags', {}).keys()) if 'tags' in app_data else 'N/A'
            genre = ', '.join(app_data.get('genres', [])) if 'genres' in app_data else 'N/A'
            return tags, genre
    return 'N/A', 'N/A'

# Lists to store the extracted tags and genre
tags_list = []
genre_list = []

# Loop through each appid in the DataFrame
for appid in tqdm(indie_games_df['appid']):
    # Fetch Tags and Genre for each game
    tags, genre = fetch_tags_and_genre(appid)
    tags_list.append(tags)
    genre_list.append(genre)

# Add the new details to the DataFrame
indie_games_df['Tags'] = tags_list
indie_games_df['Genre'] = genre_list

# Save the updated DataFrame to the same Excel file
output_file = '/content/drive/My Drive/df_spy.xlsx'  # Update the path accordingly
indie_games_df.to_excel(output_file, index=False)

print(f"Data has been saved to {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 6957/6957 [2:19:00<00:00,  1.20s/it]


Data has been saved to /content/drive/My Drive/Indie_Games_Final_Dataset.xlsx


**C. Cleaning "Genre" column**

In [ ]:
# Import necessary libraries
import pandas as pd
import re

# Load the dataset from the specified path
file_path = '/content/drive/My Drive/df_spy.xlsx'
df = pd.read_excel(file_path)

# Define the function to clean and reconstruct genres
def reconstruct_genres_with_spaces(genre_str):
    if isinstance(genre_str, str):
        # Remove commas and unnecessary spaces between letters
        cleaned_genre = genre_str.replace(', ', '').strip()

        # Combine characters into proper words
        combined_genre = ''.join(cleaned_genre).replace(' ', '')

        # List of multi-word genres that should remain intact
        multi_word_genres = [
            'Early Access', 'Massively Multiplayer', 'Free To Play', 'Open World',
            'Survival Horror', 'Animation & Modeling', 'Design & Illustration',
            'Photo Editing', 'Utilities'
        ]

        # Insert spaces between multi-word genres where needed
        for multi_genre in multi_word_genres:
            combined_genre = combined_genre.replace(multi_genre.replace(' ', '').replace('&', ''), multi_genre)

        # Return the reconstructed genres, with commas followed by spaces for readability
        return combined_genre.replace(',', ', ')

# Apply the genre reconstruction function to clean the 'Genre' column
df['Genre_final_corrected'] = df['Genre'].apply(reconstruct_genres_with_spaces)

# Save the cleaned dataset back to an Excel file
cleaned_file_path = '/content/drive/My Drive/df_spy.xlsx'
df.to_excel(cleaned_file_path, index=False)

# Download the cleaned file to your local machine
from google.colab import files
files.download(cleaned_file_path)

# Optionally, display the first few rows of the cleaned dataset to verify
df[['Genre', 'Genre_final_corrected']].head()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Genre,Genre_final_corrected
0,"A, c, t, i, o, n, ,, , A, d, v, e, n, t, u, r...","Action, Adventure, Indie, RPG, Early Access"
1,"A, c, t, i, o, n, ,, , A, d, v, e, n, t, u, r...","Action, Adventure, Indie, Massively Multiplaye..."
2,"A, c, t, i, o, n, ,, , A, d, v, e, n, t, u, r...","Action, Adventure, Casual, Indie, Free To Play"
3,"A, c, t, i, o, n, ,, , A, d, v, e, n, t, u, r...","Action, Adventure, Indie, RPG"
4,"C, a, s, u, a, l, ,, , I, n, d, i, e, ,, , S...","Casual, Indie, Simulation"


**D. Scrapping Peak Time from Steamcharts**

In [ ]:
# Step 1: Load the full dataset from the CSV file
indie_games_df = pd.read_csv('/content/drive/My Drive/df_spy.xlsx')

# Initialize list to store the all-time peak data
peak_player_data = []

# Step 2: Loop through each app_id in the full dataset and scrape all-time peak player data
for app_id in tqdm(indie_games_df['appid'], desc="Scraping SteamCharts data"):
    url = f'https://steamcharts.com/app/{app_id}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')

    # Find the div containing the "all-time peak" text
    all_time_peak_div = soup.find_all('div', class_='app-stat')

    all_time_peak = None

    # Loop through the divs and find the one with the "all-time peak" label
    for div in all_time_peak_div:
        if div.text.strip().endswith("all-time peak"):
            all_time_peak = div.find('span', class_='num').text.strip()
            break

    # Append data to the list
    peak_player_data.append({'appid': app_id, 'all_time_peak': all_time_peak})

    # Add delay to avoid too many requests in a short period
    time.sleep(1)

# Step 3: Convert the peak player data to a DataFrame
peak_players_df = pd.DataFrame(peak_player_data)

# Step 4: Merge the peak player data with the original indie_games_df
merged_df = indie_games_df.merge(peak_players_df, on='appid', how='left')

# Step 5: Save the merged dataframe with peak player data to a new CSV file
output_file_path = '/content/drive/My Drive/df_spy.xlsx'
merged_df.to_csv(output_file_path, index=False)

print(f"File saved at: {output_file_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/df_spy.xlsx'

**E. Scrapping reviews and DLC directly from Steam marketplace**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from google.colab import drive
import time  # Importing time module for sleep

drive.mount('/content/drive')

# Function to scrape overall reviews, review summary, DLC count, achievement count, and DLC names
def scrape_game_details(appid):
    url = f"https://store.steampowered.com/app/{appid}/"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            # Find the overall review summary (e.g., 'Mostly Positive')
            review_summary_elem = soup.find('div', class_='user_reviews_summary_bar').find('span', class_='game_review_summary')
            overall_review_summary = review_summary_elem.text.strip() if review_summary_elem else 'N/A'

            # Find the total number of overall reviews (e.g., "(88,649 reviews)")
            total_reviews_elem = review_summary_elem.find_next('span')
            if total_reviews_elem:
                total_reviews_text = total_reviews_elem.text.strip()
                total_reviews = int(''.join(filter(str.isdigit, total_reviews_text)))
            else:
                total_reviews = 0

            # Scrape achievements count if present
            achievements_section = soup.find('div', {'class': 'block_title'})
            achievement_count = int(achievements_section.text.strip().split()[0]) if achievements_section and 'Achievements' in achievements_section.text else 0

            return overall_review_summary, total_reviews, achievement_count

        except Exception as e:
            print(f"Error scraping for appid {appid}: {e}")
            return 'N/A', 0, 0
    return 'N/A', 0, 0

# Function to scrape DLCs from Steam Store page and return both DLC count and names
def scrape_dlcs(appid):
    url = f"https://store.steampowered.com/app/{appid}/"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            # Locate the DLC section
            dlc_section = soup.find('div', {'id': 'gameAreaDLCSection'})
            if dlc_section:
                dlc_items = dlc_section.find_all('a', {'class': 'game_area_dlc_row'})

                # List to store DLC details
                dlc_list = []

                for dlc in dlc_items:
                    dlc_name = dlc.find('div', {'class': 'game_area_dlc_name'}).text.strip()
                    dlc_list.append(dlc_name)

                # Return the list of DLCs
                return len(dlc_list), dlc_list  # DLC count and names as a list
            else:
                return 0, []  # Return empty list if no DLC section is found
        except Exception as e:
            print(f"Error scraping DLCs for appid {appid}: {e}")
            return 0, []

    return 0, []

# Function to process each appid from the DataFrame
def process_appids(df):
    review_summary_list = []
    total_reviews_list = []
    achievement_count_list = []
    dlc_count_list = []
    dlc_names_list = []

    for appid in tqdm(df['appid']):
        # Scrape overall review summary, total reviews, and achievements
        overall_review_summary, total_reviews, achievement_count = scrape_game_details(appid)
        review_summary_list.append(overall_review_summary)
        total_reviews_list.append(total_reviews)
        achievement_count_list.append(achievement_count)

        # Scrape DLC count and names
        dlc_count, dlc_names = scrape_dlcs(appid)
        dlc_count_list.append(dlc_count)
        dlc_names_list.append(', '.join(dlc_names) if dlc_names else 'No DLC')

        # Adding 1-second delay between each request
        time.sleep(1)

    # Add scraped data to the DataFrame
    df['Overall Review Summary'] = review_summary_list
    df['Total Reviews'] = total_reviews_list
    df['Achievement Count'] = achievement_count_list
    df['DLC Count'] = dlc_count_list
    df['DLC Names'] = dlc_names_list

    return df


file_path = '/content/drive/My Drive/df_spy.xlsx'
df = pd.read_excel(file_path)

# Process the DataFrame to scrape all details
df_final = process_appids(df)

# Save the processed DataFrame to Google Drive
output_file = '/content/drive/My Drive/df_spy.xlsx'
df_final.to_excel(output_file, index=False)

print(f"DataFrame saved to {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 6956/6956 [3:54:22<00:00,  2.02s/it]


DataFrame saved to /content/drive/My Drive/df_final.xlsx


**F. Scrapping achievements from Steam marketpages directly**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from google.colab import drive
import time

drive.mount('/content/drive')

# Function to scrape achievements from the Steam Store page
def scrape_achievements(appid):
    url = f"https://store.steampowered.com/app/{appid}/"
    response = requests.get(url)
    time.sleep(1)  # Add a delay to avoid being blocked by the server

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        try:
            # Scrape achievements count if present
            achievements_section = soup.find('div', {'id': 'achievement_block'})
            achievement_count = int(achievements_section.text.strip().split()[1]) if achievements_section and 'Achievements' in achievements_section.text else 0

            return achievement_count

        except Exception as e:
            print(f"Error scraping achievements for appid {appid}: {e}")
            return 0

    return 0

# Function to process each appid from the DataFrame
def process_appids_for_achievements(df):
    achievement_count_list = []

    for appid in tqdm(df['appid']):
        # Scrape achievements count
        achievement_count = scrape_achievements(appid)
        achievement_count_list.append(achievement_count)

    # Add scraped data to the DataFrame
    df['Achievement Count'] = achievement_count_list

    return df

# Load your DataFrame with appid column from Google Drive
file_path = '/content/drive/My Drive/df_spy.xlsx'  # Update with the correct path
df = pd.read_excel(file_path)

# Process the DataFrame to scrape achievements
df_final = process_appids_for_achievements(df)

# Save the processed DataFrame to Google Drive
output_file = '/content/drive/My Drive/df_final.xlsx'
df_final.to_excel(output_file, index=False)

print(f"DataFrame saved to {output_file}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


100%|██████████| 6956/6956 [2:52:25<00:00,  1.49s/it]


DataFrame saved to /content/drive/My Drive/df_finaal.xlsx
